In [ ]:
%load_ext autoreload
%autoreload 2
from ecodonut.eco_frame import EcoFrameCalculator
from ecodonut.eco_frame import EcoFrame
import geopandas as gpd
import pickle
from ecodonut.eco_frame import mark_territory

In [ ]:
# Чтение исходных геоданных (ЖД, свалки, природные резервы, промышленные зоны, лесные массивы и водные объекты)

zhd = gpd.read_file('example_data/railway_rail.geojson')
dumps = gpd.read_file('example_data/landfill.geojson')
nature_reserve = gpd.read_file('example_data/nature_reserve.geojson')
industrial = gpd.read_file('example_data/industrial.geojson')
wood = gpd.read_file('example_data/wood.geojson')
water = gpd.read_file('example_data/water.geojson')

# Загрузка границ региона
borders = gpd.read_file('example_data/region.geojson')

In [ ]:
# Формирование словаря слоёв
layers = {
    'industrial': industrial,
    "gas_station": None,
    "landfill": dumps,
    'regional_roads': None,
    'federal_roads': None,
    "railway": zhd,
    "nature_reserve": nature_reserve,
    "water": water,
    'woods': wood
}
# Создание экологического каркаса с заданием границ
frame: EcoFrame = EcoFrameCalculator(territory=borders).evaluate_ecoframe(layers, min_layer_count=2, max_layer_count=8)


In [ ]:
# Сохранение объекта EcoFrame в файл с помощью pickle
pickle.dump(frame, open('tumen.pickle', 'wb'))
# Загрузка сохранённого объекта EcoFrame из файла
eco_frame: EcoFrame = pickle.load(open('tumen.pickle', 'rb'))

In [ ]:
# Загрузка зоны для оценки территории
zone = gpd.read_file('example_data/test_borders.geojson')
# Оценка территории
mark = mark_territory(eco_frame, zone.iloc[2:3])
mark

In [ ]:
from ecodonut.eco_frame import concat_ecoframes
# Пример добавления объекта через создание нового эко-каркаса и объединения с основным
azs = gpd.read_file('example_data/fuel.geojson')
borders = gpd.read_file('example_data/region.geojson')
layers = {
    "gas_station": azs,
}
frame_add: EcoFrame = EcoFrameCalculator(territory=borders, settings_from=eco_frame).evaluate_ecoframe(layers)
new_frame = concat_ecoframes(eco_frame, frame_add)

In [ ]:
new_frame.eco_layers_gdf.explore()

## Антропогенное воздействие по почве от предприятий с классом опасности

In [ ]:
from ecodonut.eco_frame import LayerOptions


industrial = gpd.read_file('example_data/industrial.geojson')
layers = {
    'industrial': industrial,
}
def _industrial_danglvl_to_init(dang_lvl: int):
    init_values = {1: -10, 2: -8, 3: -6, 4: -4}
    return init_values.get(dang_lvl, -2)


def _industrial_danglvl_to_fading(dang_lvl: int):
    fading_values = {1: 0.2, 2: 0.1, 3: 0.05, 4: 0.01}
    return fading_values.get(dang_lvl, 0.005)

def _calculate_impact(impact_list: list) -> float:
    if len(impact_list) == 1:
        return impact_list[0]
    return min(impact_list)


layers_options = {
    "industrial": LayerOptions(
        initial_impact=("dangerous_level", _industrial_danglvl_to_init),
        fading=("dangerous_level", _industrial_danglvl_to_fading),
        russian_name="Промышленный объект",
        merge_radius=100,
    )
}
# Создание потенциального распространения загрязнения по почве от ПОО 
frame: EcoFrame = EcoFrameCalculator(territory=borders,layer_options=layers_options,impact_calculator=_calculate_impact).evaluate_ecoframe(layers, min_layer_count=2, max_layer_count=5)

In [ ]:
frame.eco_layers_gdf.explore(column='layer_impact',cmap='YlOrRd_r')